In [1]:
%load_ext rpy2.ipython

E:\ProgramData\Anaconda3\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  "The symbol '%s' is not in this R namespace/package." % name


In [2]:
%%R 
Sys.setlocale('LC_ALL', locale = "English_United States.1252") 

library(RODBC)
library(reshape2)
library(dplyr)
library(foreign)
library(readxl)
library(xlsx)
options(scipen=10)


R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [3]:
%%R
odbcCloseAll()
startd = '20210301'
endd= '20210327'
startyear<-as.numeric(substr(startd,1,4))
channel2<-odbcConnect("ShenChanBu.Fenqu",uid = "jitongbu",pwd = "xiejie")
fenqushuiliang<-sqlFetch(channel2,paste("R1Day",startyear,sep = ''))
fenqushuiliang%>%filter(.,fTagNo < 0)%>%
    mutate(.,fMonth=sprintf("%02d",fMonth),fDay=sprintf("%02d",fDay),
           QUOTA_DATE=paste(startyear,fMonth,fDay,sep=''),
           QUOTA_DEPT_CODE=fTagNo,
           #QUOTA_CODE='30984',
           QUOTA_VALUE=fData)%>%
    filter(.,QUOTA_DATE>=startd&QUOTA_DATE<=endd)%>%
    select(.,-c('fTagNo','fData','fModify','fLock'))%>%
    mutate(.,QUOTA_DEPT_CODE=-QUOTA_DEPT_CODE)->fenqushuiliang.1ji  

Area_info<-sqlFetch(channel2,'Area_info')%>%filter(.,fVaild==1)

fenqushuiliang<-merge(fenqushuiliang.1ji,Area_info,
    by.x = 'QUOTA_DEPT_CODE', by.y = 'fmain',all.y=TRUE)

fenqushuiliang%>%dcast(fMonth+fName~fDay,value.var="QUOTA_VALUE",fill = '')->fenqushuiliang

fenqushuiliang$fName <- iconv(as.vector(fenqushuiliang$fName), 'GB2312', 'UTF-8')

In [4]:
%R -o fenqushuiliang

In [5]:
fenqushuiliang

,fMonth,fName,01,02,03,04,05,06,07,08,...,18,19,20,21,22,23,24,25,26,27
1,03,北1（机场）,60943,62388,63741,64222,62496,66152,61879,65583,...,116785,113739,127555,108892,111642,127050,124533,139625,145513,149755
2,03,北2（江村东）,1047,1131,1389,1747,-731,1415,2975,7484,...,5246,3524,153,-176,2034,1945,1893,6136,8366,9188
3,03,北3（江村西）,39309,40904,40699,38886,39124,39256,34455,35310,...,40640,39116,39155,36459,39620,40882,40196,40536,40305,39678
4,03,北4（嘉禾均禾）,169980,175057,174294,188576,185517,186608,183949,206728,...,146786,146546,148671,151238,150902,139516,138133,127871,118929,140482
5,03,北5（石门石井）,341169,342048,345938,309121,295169,288059,338865,315232,...,378807,396424,402572,328476,353612,378228,383193,490534,388637,387942
6,03,北6（机场路广园新村）,253623,237935,237741,238431,239148,235428,248908,238535,...,246806,243120,233817,237015,226930,224269,229519,238661,241375,238849
7,03,北7（白云新城）,33641,31665,31511,29499,33650,32590,36117,28021,...,33820,36753,32381,29282,29283,33252,31130,35890,41170,41096
8,03,北区,899712,891128,895313,870482,854373,849508,907148,896893,...,968890,979222,984304,891186,914023,945142,948597,1079253,984295,1006990
9,03,东1(体育中心）,307339,308319,304161,306396,274676,283889,291193,316128,...,318391,321194,305475,300285,306441,303377,314869,323274,317658,316335
10,03,东10（新塘）,89667,94443,115266,116570,113892,107063,109894,105872,...,113930,113059,117488,110904,114479,113862,112376,105982,121305,115893


In [6]:
fenqushuiliang.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 1 to 38
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   fMonth  38 non-null     object
 1   fName   38 non-null     object
 2   01      38 non-null     object
 3   02      38 non-null     object
 4   03      38 non-null     object
 5   04      38 non-null     object
 6   05      38 non-null     object
 7   06      38 non-null     object
 8   07      38 non-null     object
 9   08      38 non-null     object
 10  09      38 non-null     object
 11  10      38 non-null     object
 12  11      38 non-null     object
 13  12      38 non-null     object
 14  13      38 non-null     object
 15  14      38 non-null     object
 16  15      38 non-null     object
 17  16      38 non-null     object
 18  17      38 non-null     object
 19  18      38 non-null     object
 20  19      38 non-null     object
 21  20      38 non-null     object
 22  21      38 non-null     object
 2